In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import matplotlib.image as mpimg
import matplotlib
from CNN_data_preprocess import get_label_mapping, get_files, bottle_neck_process
slim = tf.contrib.slim

In [2]:
checkpoint_file = './model_file/inception_resnet_v2_2016_08_30.ckpt'
label_file = './labels.txt'
train_file = './Data3_4/'
bottle_neck_file = './middle_data/bottle_neck3_4/'
save_model_file = './model_file/inception_resnet_v2_for3_4'
last_layer_file = './middle_data/last_layer3_4/'

In [3]:
def build_graph():
    images = tf.placeholder(tf.float32, shape=(None, 480, 720, 3))
    labels = tf.placeholder(tf.int32, shape=(None,))

    # restore from the inception_resnet model
    with slim.arg_scope(inception_resnet_v2_arg_scope()):
            _, end_points = inception_resnet_v2(images, num_classes = 1001, is_training = False) 

    # extract the before_logit tensor from the extracted model
    before_logit = end_points['PreLogitsFlatten']


    # define a placeholder for training from the bottle, not from the beginning.
    bottle_neck = tf.placeholder(tf.float32, shape=(None,1536))

    # now define the fine tune part of the graph
    with tf.name_scope('my_fine_tune') as scope:
    #     dropout = tf.layers.dropout(bottle_neck, rate=0.3, name='dropout')
        batch_norm1 = tf.layers.batch_normalization(bottle_neck, name='batch_norm1')
        dropout1 = tf.layers.dropout(batch_norm1, rate=0.3, name='dropout1')
        dense1 = tf.layers.dense(dropout1, 128, activation=tf.nn.relu, name='dense1')
        batch_norm2 = tf.layers.batch_normalization(dense1, name='batch_norm2')
        dropout2 = tf.layers.dropout(batch_norm2, rate = 0.3, name='dropout2')
        logits = tf.layers.dense(dropout2, 4, name='logits')

        x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
        loss = tf.reduce_mean(x_entropy, name='loss')
        correct = tf.nn.in_top_k(logits, labels, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

        # extract variables that need to be trained
        weight1, bias1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'dense1')
        weight2, bias2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')
        variables_to_train = [weight1, weight2, bias1, bias2]

        optimizer = tf.train.AdamOptimizer()
        train_op = slim.learning.create_train_op(loss, optimizer, variables_to_train=variables_to_train)

    # defiine the name scope we don't need to restore from inception_resnet
    exclude = ['InceptionResnetV2/AuxLogits', 'InceptionResnetV2/Logits',
               'my_fine_tune/','dense1','logits','batch_norm']

    variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
    saver = tf.train.Saver(variables_to_restore)
    init = tf.global_variables_initializer()
    builder = tf.saved_model.builder.SavedModelBuilder(save_model_file)

    return images, labels, before_logit, bottle_neck, loss, accuracy, train_op, saver, init, builder


In [4]:
# extract the data before the fine tune layer
def get_bottle_neck_data(sess, before_logit, images, train_file, bottle_neck_file, label_file, hsv = False):
    id2label, label2id = get_label_mapping(label_file)
    for i in id2label:
        read_dir = train_file + i
        des_dir = bottle_neck_file + i + '/'
        os.mkdir(des_dir)
        
        files = get_files(read_dir)
        for f in files:
            img=mpimg.imread(f).reshape(1,480,720,3) / 255
            if(hsv):
                img = matplotlib.colors.rgb_to_hsv(img)
            bottle = sess.run(before_logit, feed_dict={images:img})
            np.save(des_dir + f[len(read_dir):],  bottle)
            print('save the bottle neck file ' + f)
            

In [5]:
def get_last_layer(sess, bottle_neck_file, last_layer_file, label_file, bottle_neck, batch_norm2):
    id2label, label2id = get_label_mapping(label_file)
    for i in id2label:
        read_dir = bottle_neck_file + i
        des_dir = last_layer_file + i + '/'
        print(des_dir)
        os.mkdir(des_dir)
        
        files = get_files(read_dir)
        for f in files:
            img=np.load(f)
            print(f)
            bottle = sess.run(batch_norm2, feed_dict={bottle_neck:img})
            np.save(des_dir + f[len(read_dir):],  bottle)
            print('save the bottle neck file ' + f)

In [6]:
def train_graph(sess, model_list):
    images, labels, before_logit, bottle_neck, loss, accuracy, train_op, saver, init, builder = model_list
    
    sess.run(init)
    saver.restore(sess, checkpoint_file)

    # if there is no dir for the bottle data, then get the data first.
    if(os.path.isdir(bottle_neck_file) is not True):
        os.mkdir(bottle_neck_file)
        get_bottle_neck_data(sess, before_logit, images, train_file, bottle_neck_file, label_file, hsv = False)

    # get training data from bottle_neck file, and train.
    bottle_neck_processor = bottle_neck_process(bottle_neck_file, label_file) 
    for i in range(2000):
        inputs, groud_truth = bottle_neck_processor.next_batch(512)
        train_loss, train_accuracy = sess.run([train_op, accuracy], feed_dict={bottle_neck: inputs, 
                                                                             labels: groud_truth})

        if(i % 50 == 0):
            val_loss, val_accuracy = sess.run([loss, accuracy],
                                              feed_dict={bottle_neck: bottle_neck_processor.val_inputs,
                                                         labels: bottle_neck_processor.val_labels})
        
            print('####################################')
            print('the training loss after ' + str(i) + ' iterations is: ' + str(train_loss))
            print('the training accuracy after ' + str(i) + ' iterations is: ' + str(train_accuracy))
            print('the val loss after ' + str(i) + ' iterations is: ' + str(val_loss))
            print('the val accuracy after ' + str(i) + ' iterations is: ' + str(val_accuracy))
            print('-------------')
            
            for i in bottle_neck_processor.id2label:
                val_accuracy = sess.run([accuracy],
                                      feed_dict={bottle_neck: bottle_neck_processor.val_inputs_each[i],
                                                 labels: bottle_neck_processor.val_labels_each[i]})
                print(i + ': the val accuracy is: '  + str(val_accuracy))
        
#         after the training process finished, store the model to a '.pb' version file.
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.TRAINING],
                                         signature_def_map=None, assets_collection=None)
    
    builder.save()

In [7]:
if __name__ == '__main__':
    config = tf.ConfigProto()
    config.allow_soft_placement = True
    with tf.Session(config=config) as sess:
        model_list = build_graph()
        train_graph(sess, model_list)

INFO:tensorflow:Restoring parameters from ./model_file/inception_resnet_v2_2016_08_30.ckpt
save the bottle neck file ./Data3_4/A/a_10.jpg
save the bottle neck file ./Data3_4/A/a_1005.jpg
save the bottle neck file ./Data3_4/A/a_1007.jpg
save the bottle neck file ./Data3_4/A/a_1009.jpg
save the bottle neck file ./Data3_4/A/a_101.jpg
save the bottle neck file ./Data3_4/A/a_1010.jpg
save the bottle neck file ./Data3_4/A/a_1011.jpg
save the bottle neck file ./Data3_4/A/a_1016.jpg
save the bottle neck file ./Data3_4/A/a_1017.jpg
save the bottle neck file ./Data3_4/A/a_1024.jpg
save the bottle neck file ./Data3_4/A/a_1026.jpg
save the bottle neck file ./Data3_4/A/a_1028.jpg
save the bottle neck file ./Data3_4/A/a_1030.jpg
save the bottle neck file ./Data3_4/A/a_1034.jpg
save the bottle neck file ./Data3_4/A/a_1043.jpg
save the bottle neck file ./Data3_4/A/a_1046.jpg
save the bottle neck file ./Data3_4/A/a_1047.jpg
save the bottle neck file ./Data3_4/A/a_1048.jpg
save the bottle neck file ./Da

save the bottle neck file ./Data3_4/A/a_221.jpg
save the bottle neck file ./Data3_4/A/a_224.jpg
save the bottle neck file ./Data3_4/A/a_225.jpg
save the bottle neck file ./Data3_4/A/a_229.jpg
save the bottle neck file ./Data3_4/A/a_230.jpg
save the bottle neck file ./Data3_4/A/a_231.jpg
save the bottle neck file ./Data3_4/A/a_239.jpg
save the bottle neck file ./Data3_4/A/a_248.jpg
save the bottle neck file ./Data3_4/A/a_249.jpg
save the bottle neck file ./Data3_4/A/a_25.jpg
save the bottle neck file ./Data3_4/A/a_251.jpg
save the bottle neck file ./Data3_4/A/a_253.jpg
save the bottle neck file ./Data3_4/A/a_254.jpg
save the bottle neck file ./Data3_4/A/a_259.jpg
save the bottle neck file ./Data3_4/A/a_26.jpg
save the bottle neck file ./Data3_4/A/a_266.jpg
save the bottle neck file ./Data3_4/A/a_272.jpg
save the bottle neck file ./Data3_4/A/a_274.jpg
save the bottle neck file ./Data3_4/A/a_275.jpg
save the bottle neck file ./Data3_4/A/a_276.jpg
save the bottle neck file ./Data3_4/A/a_27

save the bottle neck file ./Data3_4/A/a_654 2.jpg
save the bottle neck file ./Data3_4/A/a_654.jpg
save the bottle neck file ./Data3_4/A/a_656.jpg
save the bottle neck file ./Data3_4/A/a_657.jpg
save the bottle neck file ./Data3_4/A/a_658.jpg
save the bottle neck file ./Data3_4/A/a_659.jpg
save the bottle neck file ./Data3_4/A/a_66.jpg
save the bottle neck file ./Data3_4/A/a_660.jpg
save the bottle neck file ./Data3_4/A/a_663.jpg
save the bottle neck file ./Data3_4/A/a_667.jpg
save the bottle neck file ./Data3_4/A/a_674.jpg
save the bottle neck file ./Data3_4/A/a_68 2.jpg
save the bottle neck file ./Data3_4/A/a_68.jpg
save the bottle neck file ./Data3_4/A/a_682.jpg
save the bottle neck file ./Data3_4/A/a_688.jpg
save the bottle neck file ./Data3_4/A/a_692.jpg
save the bottle neck file ./Data3_4/A/a_693.jpg
save the bottle neck file ./Data3_4/A/a_695.jpg
save the bottle neck file ./Data3_4/A/a_696.jpg
save the bottle neck file ./Data3_4/A/a_706.jpg
save the bottle neck file ./Data3_4/A/a

save the bottle neck file ./Data3_4/D/d_1170 2.jpg
save the bottle neck file ./Data3_4/D/d_1170.jpg
save the bottle neck file ./Data3_4/D/d_1171.jpg
save the bottle neck file ./Data3_4/D/d_1174 2.jpg
save the bottle neck file ./Data3_4/D/d_1174.jpg
save the bottle neck file ./Data3_4/D/d_1178.jpg
save the bottle neck file ./Data3_4/D/d_118.jpg
save the bottle neck file ./Data3_4/D/d_1182.jpg
save the bottle neck file ./Data3_4/D/d_1188 2.jpg
save the bottle neck file ./Data3_4/D/d_1188.jpg
save the bottle neck file ./Data3_4/D/d_1193.jpg
save the bottle neck file ./Data3_4/D/d_1196.jpg
save the bottle neck file ./Data3_4/D/d_1197 2.jpg
save the bottle neck file ./Data3_4/D/d_1197.jpg
save the bottle neck file ./Data3_4/D/d_1198.jpg
save the bottle neck file ./Data3_4/D/d_120.jpg
save the bottle neck file ./Data3_4/D/d_1200.jpg
save the bottle neck file ./Data3_4/D/d_1205.jpg
save the bottle neck file ./Data3_4/D/d_1212.jpg
save the bottle neck file ./Data3_4/D/d_1217.jpg
save the bottl

save the bottle neck file ./Data3_4/D/d_1534.jpg
save the bottle neck file ./Data3_4/D/d_1535 2.jpg
save the bottle neck file ./Data3_4/D/d_1535.jpg
save the bottle neck file ./Data3_4/D/d_1536.jpg
save the bottle neck file ./Data3_4/D/d_1537 2.jpg
save the bottle neck file ./Data3_4/D/d_1537.jpg
save the bottle neck file ./Data3_4/D/d_1538.jpg
save the bottle neck file ./Data3_4/D/d_1539.jpg
save the bottle neck file ./Data3_4/D/d_154.jpg
save the bottle neck file ./Data3_4/D/d_1541.jpg
save the bottle neck file ./Data3_4/D/d_1542.jpg
save the bottle neck file ./Data3_4/D/d_1543.jpg
save the bottle neck file ./Data3_4/D/d_1544.jpg
save the bottle neck file ./Data3_4/D/d_1547.jpg
save the bottle neck file ./Data3_4/D/d_1549.jpg
save the bottle neck file ./Data3_4/D/d_1550.jpg
save the bottle neck file ./Data3_4/D/d_156.jpg
save the bottle neck file ./Data3_4/D/d_1567.jpg
save the bottle neck file ./Data3_4/D/d_1586.jpg
save the bottle neck file ./Data3_4/D/d_1587 2.jpg
save the bottle 

...

save the bottle neck file ./Data3_4/W/w_672.jpg
save the bottle neck file ./Data3_4/W/w_673 2.jpg
save the bottle neck file ./Data3_4/W/w_673.jpg
save the bottle neck file ./Data3_4/W/w_674.jpg
save the bottle neck file ./Data3_4/W/w_675.jpg
save the bottle neck file ./Data3_4/W/w_676 2.jpg
save the bottle neck file ./Data3_4/W/w_676.jpg
save the bottle neck file ./Data3_4/W/w_677 2.jpg
save the bottle neck file ./Data3_4/W/w_677.jpg
save the bottle neck file ./Data3_4/W/w_678 2.jpg
save the bottle neck file ./Data3_4/W/w_678.jpg
save the bottle neck file ./Data3_4/W/w_679 2.jpg
save the bottle neck file ./Data3_4/W/w_679.jpg
save the bottle neck file ./Data3_4/W/w_680 2.jpg
save the bottle neck file ./Data3_4/W/w_680.jpg
save the bottle neck file ./Data3_4/W/w_681 2.jpg
save the bottle neck file ./Data3_4/W/w_681.jpg
save the bottle neck file ./Data3_4/W/w_682.jpg
save the bottle neck file ./Data3_4/W/w_683.jpg
save the bottle neck file ./Data3_4/W/w_684 2.jpg
save the bottle neck fil

save the bottle neck file ./Data3_4/W/w_771 2.jpg
save the bottle neck file ./Data3_4/W/w_771.jpg
save the bottle neck file ./Data3_4/W/w_772 2.jpg
save the bottle neck file ./Data3_4/W/w_772.jpg
save the bottle neck file ./Data3_4/W/w_773 2.jpg
save the bottle neck file ./Data3_4/W/w_773.jpg
save the bottle neck file ./Data3_4/W/w_774.jpg
save the bottle neck file ./Data3_4/W/w_775 2.jpg
save the bottle neck file ./Data3_4/W/w_775.jpg
save the bottle neck file ./Data3_4/W/w_776.jpg
save the bottle neck file ./Data3_4/W/w_777 2.jpg
save the bottle neck file ./Data3_4/W/w_777.jpg
save the bottle neck file ./Data3_4/W/w_778 2.jpg
save the bottle neck file ./Data3_4/W/w_778.jpg
save the bottle neck file ./Data3_4/W/w_779 2.jpg
save the bottle neck file ./Data3_4/W/w_779.jpg
save the bottle neck file ./Data3_4/W/w_78 2.jpg
save the bottle neck file ./Data3_4/W/w_78.jpg
save the bottle neck file ./Data3_4/W/w_781.jpg
save the bottle neck file ./Data3_4/W/w_782.jpg
save the bottle neck file 

save the bottle neck file ./Data3_4/W/w_883 2.jpg
save the bottle neck file ./Data3_4/W/w_883.jpg
save the bottle neck file ./Data3_4/W/w_884 2.jpg
save the bottle neck file ./Data3_4/W/w_884.jpg
save the bottle neck file ./Data3_4/W/w_885.jpg
save the bottle neck file ./Data3_4/W/w_886.jpg
save the bottle neck file ./Data3_4/W/w_887 2.jpg
save the bottle neck file ./Data3_4/W/w_887.jpg
save the bottle neck file ./Data3_4/W/w_889 2.jpg
save the bottle neck file ./Data3_4/W/w_889.jpg
save the bottle neck file ./Data3_4/W/w_89 2.jpg
save the bottle neck file ./Data3_4/W/w_89.jpg
save the bottle neck file ./Data3_4/W/w_890.jpg
save the bottle neck file ./Data3_4/W/w_891.jpg
save the bottle neck file ./Data3_4/W/w_892 2.jpg
save the bottle neck file ./Data3_4/W/w_892.jpg
save the bottle neck file ./Data3_4/W/w_893.jpg
save the bottle neck file ./Data3_4/W/w_894.jpg
save the bottle neck file ./Data3_4/W/w_895.jpg
save the bottle neck file ./Data3_4/W/w_896 2.jpg
save the bottle neck file ./

save the bottle neck file ./Data3_4/W/w_989.jpg
save the bottle neck file ./Data3_4/W/w_99.jpg
save the bottle neck file ./Data3_4/W/w_990 2.jpg
save the bottle neck file ./Data3_4/W/w_990.jpg
save the bottle neck file ./Data3_4/W/w_991 2.jpg
save the bottle neck file ./Data3_4/W/w_991.jpg
save the bottle neck file ./Data3_4/W/w_992 2.jpg
save the bottle neck file ./Data3_4/W/w_992.jpg
save the bottle neck file ./Data3_4/W/w_993 2.jpg
save the bottle neck file ./Data3_4/W/w_993.jpg
save the bottle neck file ./Data3_4/W/w_994 2.jpg
save the bottle neck file ./Data3_4/W/w_994.jpg
save the bottle neck file ./Data3_4/W/w_995 2.jpg
save the bottle neck file ./Data3_4/W/w_995.jpg
save the bottle neck file ./Data3_4/W/w_996 2.jpg
save the bottle neck file ./Data3_4/W/w_996.jpg
save the bottle neck file ./Data3_4/W/w_997 2.jpg
save the bottle neck file ./Data3_4/W/w_997.jpg
save the bottle neck file ./Data3_4/W/w_998.jpg
save the bottle neck file ./Data3_4/W/w_999.jpg
./middle_data/bottle_neck

####################################
the training loss after 1000 iterations is: 0.315615
the training accuracy after 1000 iterations is: 0.878431
the val loss after 1000 iterations is: 0.472608
the val accuracy after 1000 iterations is: 0.810484
-------------
A: the val accuracy is: [0.63953495]
D: the val accuracy is: [0.51181102]
W: the val accuracy is: [0.90960449]
####################################
the training loss after 1050 iterations is: 0.272422
the training accuracy after 1050 iterations is: 0.901961
the val loss after 1050 iterations is: 0.486212
the val accuracy after 1050 iterations is: 0.795699
-------------
A: the val accuracy is: [0.66279078]
D: the val accuracy is: [0.55118108]
W: the val accuracy is: [0.8757062]
####################################
the training loss after 1100 iterations is: 0.275636
the training accuracy after 1100 iterations is: 0.9
the val loss after 1100 iterations is: 0.4881
the val accuracy after 1100 iterations is: 0.797043
-------------
A: 